In [ ]:
!pip install transformers
!pip install datasets
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, DataCollatorForLanguageModeling, TrainingArguments, Trainer
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2", bos_token="<s>", eos_token="</s>")
model = AutoModelForCausalLM.from_pretrained(r"/content/drive/MyDrive/Model", use_cache=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
dataset = load_dataset("csv", data_files={"train": r"/content/train_dataset.csv", "test" : r"/content/test_dataset.csv"})
dataset

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2082
    })
    test: Dataset({
        features: ['text'],
        num_rows: 520
    })
})

In [ ]:
block_size = 2048
def tokenize_function(examples):
    return tokenizer(examples["text"])

def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i: i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    return result

column_names = dataset["train"].column_names
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=column_names,
)
result_dataset = tokenized_datasets.map(group_texts, batched=True)

In [ ]:
from transformers import DataCollatorForLanguageModeling
tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
    )
print(result_dataset.shape)
tokenized_datasets.shape

{'train': (33, 2), 'test': (10, 2)}


{'train': (2082, 2), 'test': (520, 2)}

In [ ]:
print(tokenizer.all_special_tokens)
tokenizer

['<s>', '</s>', '<|endoftext|>', '</s>']


PreTrainedTokenizerFast(name_or_path='sberbank-ai/rugpt3medium_based_on_gpt2', vocab_size=50257, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '</s>'})

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="test_trainer",
    overwrite_output_dir=True,
    gradient_accumulation_steps=4,
    lr_scheduler_type="cosine",
    learning_rate=5e-5,
    fp16=True,
    num_train_epochs=20,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    evaluation_strategy="steps",
    eval_steps = 100,
    save_total_limit = 3,
    load_best_model_at_end=True,
    weight_decay=0.01,
)
trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=tokenized_datasets["train"], 
    eval_dataset = tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    )
trainer.train()
trainer.save_model("/content/drive/MyDrive/TrainModel")

Using cuda_amp half precision backend
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2082
  Num Epochs = 20
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 4
  Total optimization steps = 10400
  Number of trainable parameters = 355871744
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
100,No log,6.291186
200,No log,6.320421
300,No log,6.305661
400,No log,6.286871
500,0.344900,6.143619
600,0.344900,6.217131
700,0.344900,6.157073
800,0.344900,6.155322
900,0.344900,6.211659
1000,0.360900,6.162163


***** Running Evaluation *****
  Num examples = 520
  Batch size = 1
***** Running Evaluation *****
  Num examples = 520
  Batch size = 1
***** Running Evaluation *****
  Num examples = 520
  Batch size = 1
***** Running Evaluation *****
  Num examples = 520
  Batch size = 1
***** Running Evaluation *****
  Num examples = 520
  Batch size = 1
Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 520
  Batch size = 1
***** Running Evaluation *****
  Num examples = 520
  Batch size = 1
***** Running Evaluation *****
  Num examples = 520
  Batch size = 1
***** Running Evaluation *****
  Num examples = 520
  Batch size = 1
***** Running Evaluation *****

Step,Training Loss,Validation Loss
100,No log,6.291186
200,No log,6.320421
300,No log,6.305661
400,No log,6.286871
500,0.344900,6.143619
600,0.344900,6.217131
700,0.344900,6.157073
800,0.344900,6.155322
900,0.344900,6.211659
1000,0.360900,6.162163


***** Running Evaluation *****
  Num examples = 520
  Batch size = 1
Saving model checkpoint to test_trainer/checkpoint-9500
Configuration saved in test_trainer/checkpoint-9500/config.json
Model weights saved in test_trainer/checkpoint-9500/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-9500/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-9500/special_tokens_map.json
Deleting older checkpoint [test_trainer/checkpoint-8500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 520
  Batch size = 1
***** Running Evaluation *****
  Num examples = 520
  Batch size = 1
***** Running Evaluation *****
  Num examples = 520
  Batch size = 1
***** Running Evaluation *****
  Num examples = 520
  Batch size = 1
***** Running Evaluation *****
  Num examples = 520
  Batch size = 1
Saving model checkpoint to test_trainer/checkpoint-10000
Configuration saved in test_trainer/checkpoint-10000/config.json
Model weights saved in t